In [5]:
from glob import glob
from plotly.offline import init_notebook_mode
import numpy as np
import pandas as pd
import plotly.graph_objects as go
import plotly.io as pio
from astropy.timeseries import LombScargle
from ipywidgets import interact, widgets
from plotly.subplots import make_subplots
init_notebook_mode(connected=True)

data_file_widget = widgets.Dropdown(
    options=glob("./*.dat"),
    description="Mesaurement File:",
    style=dict(description_width="initial")
)
theme_widget = widgets.Dropdown(
    options=pio.templates,
    description="Theme:",
    style=dict(description_width="initial")
)
power_normalization_widget = widgets.Dropdown(
    options=["standard", "model", "log", "psd"],
    description="Power Normalization:",
    style=dict(description_width="initial")
)
autopower_method_widget = widgets.Dropdown(
    options=["auto", "fast", "slow", "cython", "chi2", "fastchi2", "scipy"],
    description="Autopower Method:",
    style=dict(description_width="initial")
)
min_frequency_widget = widgets.FloatText(
    description="Minimum Frequency (0 = ignore):",
    style=dict(description_width="initial")
)
max_frequency_widget = widgets.FloatText(
    description="Maximum Frequency (0 = ignore):",
    style=dict(description_width="initial")
)
nyquist_factor_widget = widgets.FloatText(
    value=5,
    description="Nyquist Factor (5 = default):",
    style=dict(description_width="initial")
)


@interact
def display_lomb_scargle(data_file=data_file_widget,
                         theme=theme_widget,
                         power_normalization=power_normalization_widget,
                         autopower_method=autopower_method_widget,
                         min_frequency=min_frequency_widget,
                         max_frequency=max_frequency_widget,
                         nyquist_factor=nyquist_factor_widget):
    min_frequency = None if min_frequency == 0 else min_frequency
    max_frequency = None if max_frequency == 0 else max_frequency

    df = pd.read_csv(data_file, sep=" ", names=("time", "magnitude", "magnitude_uncertainty", "dont_worry_about_it"))
    t = df["time"]
    m = df["magnitude"]
    dm = df["magnitude_uncertainty"]

    ls = LombScargle(t, m, dm)
    frequency, power = ls.autopower(method=autopower_method, nyquist_factor=nyquist_factor,
                                    minimum_frequency=min_frequency, maximum_frequency=max_frequency, normalization=power_normalization_widget)

    best_frequency = frequency[np.argmax(power)]
    t_fit = np.linspace(0, 1)
    y_fit = ls.model(t_fit, best_frequency)

    fig = make_subplots(rows=1, cols=2)

    fig.add_trace(go.Scatter(x=frequency, y=power, name="Lomb-Scargle Power"), row=1, col=1)
    fig.update_xaxes(title_text="Frequency", row=1, col=1)
    fig.update_yaxes(title_text="Power", row=1, col=1)

    fig.add_trace(go.Scatter(x=t_fit, y=y_fit, name="Lomb-Scargle Model (using best frequency)"), row=1, col=2)
    fig.update_xaxes(title_text="Time", row=1, col=2)
    fig.update_yaxes(title_text="Magnitude", row=1, col=2)

    fig.update_layout(title_text=f"Measurements File: {data_file}", template=theme, legend=dict(
        orientation="h",
        yanchor="bottom",
        y=1.02,
        xanchor="right",
        x=1
    ))

    return fig


interactive(children=(Dropdown(description='Mesaurement File:', options=('.\\iphas0053_r.dat', '.\\iphas0053_r…